In [1]:
import pymesh
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

In [2]:
landmarks = pd.read_csv("../data_artificial_cites/landmarks_reales/model_mesh_002.pts",header=None,index_col=[0],skiprows=range(2),delimiter=r"\s+")

In [3]:
ls0 = landmarks.iloc[0].tolist()
ls1 = landmarks.iloc[4].tolist()
ls2 = landmarks.iloc[9].tolist()
ls3 = landmarks.iloc[10].tolist()
head = landmarks.iloc[14].tolist()
foot_left = landmarks.iloc[7].tolist()
foot_right = landmarks.iloc[6].tolist()

In [4]:
landmarks_torzo = np.array([ls0,ls1,ls2,ls3])
centroide = landmarks_torzo.mean(0)

In [5]:
output_mesh = pymesh.load_mesh("out/torso.ply")

In [6]:
def distance3D(a, b):
    return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)

In [7]:
semi_landmark = landmarks_torzo[1:3].mean(0)
distancias = []
for v in output_mesh.vertices:
    distancias.append(distance3D(semi_landmark,v))
distancias = np.array(distancias)
ver_semi_landmark = output_mesh.vertices[distancias.argmin()]
distancias.argmin()

18351

In [8]:
def angle(a,b):
    uv = a[0]*b[0] + a[1]*b[1]+ a[2]*b[2]
    norma_uv = math.sqrt((a[0]**2+a[1]**2+a[2]**2)) * math.sqrt((b[0]**2+b[1]**2+b[2]**2 ))
    return math.acos(uv/norma_uv)

In [9]:
new_mesh = pymesh.meshutils.generate_icosphere(0.01,ver_semi_landmark)
q = pymesh.misc.Quaternion(quat=[1,0.28, 0, 0])
#ver_semi_landmark = [0,0,0]
circle_points = []
for i in range(30):
    distance_points = 0.3
    #angleY = 0.0174533
    angleY = (i)
    angleZ = 0
    x= ver_semi_landmark[0] + distance_points * math.cos (angleZ) * math.sin(angleY)
    y= ver_semi_landmark[1] + distance_points * math.sin (angleZ)
    z= ver_semi_landmark[2] + distance_points * math.cos (angleZ) * math.cos(angleY)
    #z= ver_semi_landmark[2]
    #v = q.rotate([x,y,z])
    v_v = [x,y,z] 
    
#     distancias_circle=[]
#     for v_m in output_mesh.vertices:
#         distancias_circle.append(distance(v,v_m))
#     v_v = output_mesh.vertices[np.array(distancias_circle).argmin()]
#     q.conjugate()
    #v = q.rotate(v_v)
    v = q.rotate(v_v)
    v[2] = v[2] /5
    v[1] = v[1] /5
    v[0] = v[0] /4.9
    circle_points.append(v)
    #temp = pymesh.meshutils.generate_icosphere(0.01,v)
    #new_mesh = pymesh.boolean(new_mesh, temp , operation="union",engine="auto")
    temp = pymesh.meshutils.generate_icosphere(0.01,v)
    new_mesh = pymesh.boolean(new_mesh, temp , operation="union",engine="auto")
pymesh.save_mesh("out/circle.ply", new_mesh)

In [10]:
circle_points = np.array(circle_points)
circle_points.shape

(30, 3)

In [11]:
vertexs = []
cp = circle_points[0]
for cp in circle_points:
    distancias_circle = []
    for v in output_mesh.vertices:
        distancias_circle.append(distance3D(cp,v))
    vertexs.append(output_mesh.vertices[np.array(distancias_circle).argmin()])

In [12]:
new_mesh = pymesh.meshutils.generate_icosphere(0.01,vertexs[0])
for v in vertexs[1:]:
    temp = pymesh.meshutils.generate_icosphere(0.01,v)
    new_mesh = pymesh.boolean(new_mesh, temp , operation="union",engine="auto")
pymesh.save_mesh("out/cinta.ply", new_mesh)

In [13]:
sumatoria =[]
for i,j in zip(vertexs,vertexs[1:]):
    sumatoria.append(distance3D(i,j))
sumatoria = np.array(sumatoria)
sumatoria.sum()

4.8512765817983832

In [14]:
a = np.array(vertexs)
a1 = vertexs[0]

In [15]:
new_array = [tuple(row) for row in a]
uniques = np.unique(new_array)

In [16]:
aux = 0
for i,v in enumerate(uniques[:-1]):
    aux = aux + np.min(map(lambda x: distance3D(x,v),uniques[i+1:]))
aux

1.2294657403880505

In [17]:
foot_center = np.array([foot_left,foot_right]).mean(0)

In [18]:
foot_center

array([ 0.01473366, -0.93171081, -0.11719008])